# 🐸 <span style='color :#40be46' > JFrog Swampup 2024 MLOPs LAB - The Frog-Factor Authenticator </span> 🐸 

Welcome to the Lab! 


At JFrog, we're always exploring new features and capabilities. Today, we're diving into the authentication market with a brilliant idea: the "Frog-Factor" authenticator. This unique tool will authenticate you by recognizing your face alongside the JFrog frog in the same photo!

Your mission, if you choose to accept it, is to help us build the Frog-Factor authenticator.

First up, we'll need an object-detection model to get us started. Fortunately, we don't have to start from scratch! There are existing models we can use. But will they work out of the box? And remember, we must develop this authenticator securely and reliably.

As you work through the notebook, follow the cells in order:

✨ - This icon means there's a task for you to complete before moving to the next cell.

👀 - This icon provides information about what the next cell is doing.

Let's get started! We're here to help you every step of the way.

# <span style='color :#40be46' > Let's prepare the environment </span>


✨ <span style='color : #fae253' > TASK </span> ✨

Since we want to work in a secure and trusted way, we will use Artifactory to store both the model as well as all the python dependencies that will be used during the process. 

Please perform the following steps:
1. Log in to the training Artifactory here: https://jftrain17224572670.jfrog.io/
2. From the projects dropdown list, select your project ("mlops-userx")
3. Navigate to *Administration --> Repositories*.
4. Click on 'Create a Repository' and select *Remote*. Select *HuggingFaceML* .
5. In the next page, only provide the *Repository Key* (the repo name)  **The repository will be prefixed with your project name ("mlops-userx"). Please add the repository key "hf-remote"**  then click `Create Remote Repository`

# 🐸  <span style='color :#40be46' > Lab1: Caching HuggingFace models in Artifactory </span> 🐸 

## Configure HuggingFace client to work through Artifactory

We don't have to start from scratch! Luckily HuggingFace contains some great object detection models we can try out. 
Since we want to store the models in Artifactory, we'll need to configure the environment as follows.

✨ <span style='color : #fae253' > TASK </span> ✨

Please open your Artifactory instance and navigate to your newly created *remote HuggingFaceML* repository and click on "Set Me Up" in the top bar on the right.
1. Copy the *token* and paste it in the cell below, replacing the \<IDENTITY_TOKEN> placeholder 
2. Replace x (in *userx-hf-remote* inside the HF_ENDPOINT env var) with your lab studentId number

👀 The next cell sets the environment variables such that the huggingface client which we'll use later will not fetch the model from the hugging_face hub, but rather from Artifactory.

In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HF_TOKEN=<IDENTITY-TOKEN>

# Replace x (in userx-hf-remote) with your userid number
%env HF_ENDPOINT=https://jftrain17224572670.jfrog.io/artifactory/mlops-userx-hf-remote/

%env HF_HUB_ETAG_TIMEOUT=86400

## Download the required Python packages through Artifactory

To use and test our model, we'll need some python packages. Since we want to make sure we're using trusted and allowed packages, we'll get the packages also from Artifactory.
We'll configure the python installations to go to Artifactory to fetch the packages. 

We've already configured a pypi repository for you to use. Run the next cell to download the required dependencies. 

*NOTE:*
The following cell may take up to 3 minutes to complete. 

✨ <span style='color : #fae253' > TASK </span> ✨

1. In Artifactory,go to our pypi repository: https://jftrain17224572670.jfrog.io/ui/repos/tree/General/mlops-training-remote-pypi 
2. Click on `Set Me Up` in the top bar, and click on `Install` tab.
3. Copy the URL from the value of index-url (should look something like https://youruser:<PASSWORD>@jftrain17224572670.jfrog.io/artifactory/api/pypi/mlops-training-remote-pypi/simple)
4. Replace \<ARTIFACTORY_PIP_REPOSITORY_URL> with the URL you copied .


*NOTE:*
The following cell may take up to 3 minutes to complete. 

In [ ]:
# Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up.
!pip3 install huggingface_hub ultralytics -i <ARTIFACTORY_PIP_REPOSITORY_URL>

## Python imports

In [ ]:
from huggingface_hub import snapshot_download, HfApi
from huggingface_hub.utils import HfHubHTTPError

import json
import random
from ultralytics import YOLO

import cv2

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

from google.colab.patches import cv2_imshow

import logging,shutil

## Download the pre-trained model

👀 We'll be using the Yolov8 object detection pre-trained model. It's initially configured only to detect human faces.

✨ <span style='color : #fae253' > TASK </span> ✨

Replace \<YOUR_NAME> with your name. Don't forget to surround it by quote e.g. "Tom".

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Load the model and processor
model_name = "shirabendor/YOLOV8-oiv7"
weights = "yolov8m-oiv7.pt"
config_file = "./model/main/config.json"
name = "<YOUR_NAME>"

try:
    snapshot_download(repo_id=model_name, allow_patterns=[weights, "mlops.zip"], local_dir=".")
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the `Block Malicious Packages` policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

# unpack the other course materials and remove the default folder by colab
!unzip mlops.zip
!rm -rf sample_data

✨ <span style='color : #fae253' > TASK </span> ✨

Let's check Artifactory to review the outcome.

Please open your Artifactory instance and navigate to your newly created *remote HuggingfaceML repository*

Here is a direct link (replace x with your studentID):  https://jftrain17224572670.jfrog.io/artifactory/mlops-userx-hf-remote/

Expect to see something similar to the following:

![image.png](./model/main/img/repo.png)

## Helper Functions

👀 The following cell defines some helper functions that will help us to test and develop the models.

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename



# Inference function

👀 The following cell defines the inference function. The "predict" function will get an image as input, and try to detect human face in the image. 

In [ ]:
logging.getLogger("ultralytics").setLevel(logging.ERROR) 

model = YOLO(weights)

def infere(cheat=False, name=name):
    
    with open(config_file, 'r') as f:
        config = json.load(f)

    classes        = config['classes']
    target_classes = config['target_classes']
    conf           = config['conf']
    max_det        = config['max_det']

    filename = 'photo.jpg'  # Default filename
    if not cheat:
        filename = take_photo()
    else:
      data = "./model/main/img/tom.jpg"
      name = "Tom Hanks"
      shutil.copy(data, filename)
           

    frame = cv2.imread(filename) 

    frame_height, frame_width = frame.shape[:2]
    results = model.predict(source=frame, 
                            show=False, 
                            classes=classes, 
                            conf=conf,
                            max_det=max_det)

    # Extracting the names of detected classes
    boxes = results[0].boxes

     # Draw bounding boxes
    for box in boxes:
        label = model.names[int(box.cls)]
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convert to integer coordinates
  
        if int(box.cls) in target_classes:
          # Draw bounding box around detected object
          cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)  # Colored box 
          cv2.putText(frame, "Frog", (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        else:
          cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)  # Colored box 
          cv2.putText(frame, name, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,  (255, 0, 0), 2)

    cv2_imshow(frame)



Let's go ahead and test the model! 
Please approve using the camera for the model to work.
Once you have the video stream, click "Capture" to take a photo, and examine the model's output.
Then, run the cell again and take a photo of yourself and the Jfrog frog. Did the model identify the frog?

*NOTE:*
The following cell may fail in the first time because it runs in parallel to requesting the access to the webcam. If it happens just rerun the cell.

In [ ]:
try:
    infere()
except Exception as e:
    cv2.destroyAllWindows()
    raise e


In case you can't or don't want to use your own picture, you can use the cheat sheet cell below:

In [ ]:
try:
    infere(cheat=True)
except Exception as e:
    cv2.destroyAllWindows()
    raise e

---

# 🐸  <span style='color :#40be46' > Lab2: Securing models </span> 🐸 

## Block malicious model with Xray

Our Yolo model did quite well identifying human faces, but we wanted it to also detect frogs.
What can we do?
Searching HuggingFace, there is a model that seems just right! 

But is it safe? Let's configure Xray for our HuggingFace repository and find out. 

✨ <span style='color : #fae253' > TASK </span> ✨

Let's configure Xray to scan our HuggingFaceML remote repository.

#### Complete the following steps:

***Add the HuggingFaceML remote repository to XRay index:***

1. Navigate to *Administration --> Xray Settings --> Indexed Resources --> Repositories* and click on `+ Add a Reposotiry`
2. Search for your repository name on the right hand side search box, and drag it to the left hand side table. Click `Save`
3. Search for your repository in the repositories list, and click on the ... menu. Click "Index Now" . No need to change anything in the dialog that opens. Refresh the status to see the results.

For your convenience, we've already created a policy and a watch, so just adding your repository will be enough to kick off scanning. 
You can examine them in the following links:

1. Policy: https://jftrain17224572670.jfrog.io/ui/admin/xray/policiesGovernance/policies/edit/block-malicious-models
2. Watch: https://jftrain17224572670.jfrog.io/ui/admin/xray/policiesGovernance/watches/edit/bloack-malicious-model-watch?activeTab=violations


---

In [ ]:
try:
    snapshot_download(repo_id="MustEr/best_model_for_identifying_frogs")
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Malicious Package' policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

### Check the scanning results 

✨ <span style='color : #fae253' > TASK </span> ✨

Navigate to your project's scans list (**remembe to replace x with your student id**)

https://jftrain17224572670.jfrog.io/ui/scans-list/repositories?projectKey=mlops-userx

# 🐸  <span style='color :#40be46' > Lab3: Uploading updated model to a local repository and deploying with Qwak </span> 🐸 

## Train the model to identify Frogs

Unfortunately, the "best_model_for_identifying_frogs" was not safe and we cannot use it.
But we still want to detect the frogs. Next, we will 'train' our original Yolo model to identify other objects, specifically frogs.

👀  Due to time constraints, our training function does not actually train on additional images. Instead, we'll just change the model configuration. Check the "config.json" file before and after the training to see the difference.


In [ ]:
def train(object_to_detect):

    if not object_to_detect in model.names.values():
        print(f"'{object_to_detect}' is not a valid YOLOv8 object. Hint: try Frog")
        return

    reverse_dict = {name: idx for idx, name in model.names.items()}
    class_id = reverse_dict.get(object_to_detect, None)

    with open(config_file, 'r') as file:
        config = json.load(file)

    target_classes = config['target_classes']

    # Add the new class number to the classes list if it's not already present
    if class_id not in config['classes']:
        config['classes'].append(class_id)
        config['classes'].extend([cls for cls in target_classes if cls not in config['classes']])


    # Save the updated config back to the file
    with open(config_file, 'w') as file:
        json.dump(config, file, indent=4)


In [ ]:
train("Frog")

## Run inference again

Let's check if the training did the trick!
Please take the JFrog frog and take a photo of the two of you together 😊 🐸 

In [ ]:
try:
    infere()
except Exception as e:
    cv2.destroyAllWindows()
    raise e


Again, Cheat sheet cell is available if you like in the next cell.

In [ ]:
try:
    infere(cheat=True)
except Exception as e:
    cv2.destroyAllWindows()
    raise e


## Upload to HF local

Now that we have a new, trained model, we need to upload it to Artifactory HugginigFaceML local repository in order to share it with other teams and promote it towards Production.

✨ <span style='color : #fae253' > TASK </span> ✨

Please perform the following steps:
1. Navigate to *Administration --> Repositories*.
2. Create a **local** HuggingFaceML repository in your Artifactory project. This repository will be used to cache the models HuggingFace.
3. Navigate to "Application --> Artifactory --> Artifacts" and find your newly created local repository. Click on "Set Me Up" and the top bar on the right.
4. Copy the *token* and paste it in the cell below, replacing the <IDENTITY_TOKEN> placeholder
5. Copy the HF_ENDPOINT value and paste it in the cell below, replacing the <PATH> placeholder


In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HF_TOKEN=<IDENTITY-TOKEN>

# Replace the <PATH> placeholder with the path to your ML Model Management repository in Artifactory, found in the JFrog Platform SetMeUp.
%env HF_ENDPOINT=<PATH>

%env HF_HUB_DOWNLOAD_TIMEOUT=86400
%env HF_HUB_ETAG_TIMEOUT=86400

In [ ]:
from huggingface_hub import HfApi
import os

# Initialize API with the custom endpoint
api = HfApi(endpoint=os.getenv("HF_ENDPOINT"))

# Upload folder to the specified repository
api.upload_folder(
    folder_path=".",
    repo_id="frog-factor1",   # Replace with a name for your model
    repo_type="model"
)

### Check the results in Artifactory

✨ <span style='color : #fae253' > TASK </span> ✨

Let's check Artifactory to review the outcome.


1. Please open your Artifactory instance Navigate to *Artifactory --> Artifacts* tab.
2. Find your newly created *local HuggingFaceML repository*.
3. Expand the repository and verify the YOLOV8 model is cached inside the repository, including the updated configuration file.

## Deploy with Qwak

Now that we have a good model version, let's deploy it to a production endpoint with Qwak and moitor its performance.
Qwak A fully managed end-to-end platform that contains the infrastructure AI practitioners need to build, deploy, manage and monitor GenAI, LLMs and classic ML in production.

We'll start by installing the qwak SDK.

✨ <span style='color : #fae253' > TASK </span> ✨

1. Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up (you can take it from the cell in the first lab)
2. Create a personal API key in the Qwak platform:
    - Go to [Quak Platform](https://app.qwak.ai/)
    - On the left hand side menu, Navigate to *Settings --> Personal API Keys*
    - Click on `Generate API Key`
    - Copy the API key generated and replace the below <QWAK_PERSONAL_API_KEY> placeholder with it

In [ ]:
# Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up.
!pip3 install qwak-sdk -i <ARTIFACTORY_PIP_REPOSITORY_URL>
# Replace <QWAK_PERSONAL_API_KEY> with your Qwak personal key from the qwak platform.
!qwak configure --api-key <QWAK_PERSONAL_API_KEY>

# Test successful connection
!qwak projects list

### Build the Qwak model

✨ <span style='color : #fae253' > TASK </span> ✨

In order to build and deploy the model through the Qwak platform, we'll need to first find the model-id.
1. In the [Quak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select your project and click on your model.
3. Copy your model-id from the information bar under the title with the model name (you'll have a clickable copy icon once you hover on it)
4. Replace the <MODEL_ID> placeholder below with your model_id.


In [ ]:
!qwak models build --model-id yolo_test_1 ./model

### Check your model build status (can take up to 10 minutes)

### Deploy your model

✨ <span style='color : #fae253' > TASK </span> ✨

1. In the [Quak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select your project and click on your model.
3. Under the *Builds* tab, identify your build and click `Deploy`
4. Select `Realtime`
5. On the next screen, no need to change anything, click on `Deploy Model`


In [ ]:
models.shira-jfrog1.qwak.ai/v1/yolo_test_1/default/predict()